# Mapping electricity generation and trade to ecoinvent

To run this notebook, you must have the following environment variables set:

* BENTSO_DATA_DIR: Directory to cache data from ENTSO-E API

In addition, you need to have finished the notebook `ENTSO-E electricity data`.

In [1]:
import bw2data as bd
import bw2io as bi
import pandas as pd

In [2]:
assert bd.__version__ >= (4, 0, 'DEV11')

In [3]:
bd.projects.set_current("GSA")

## Find electricity generators in Europe

In [4]:
ei = bd.Database("ecoinvent")

In [5]:
from bentso.constants import ENTSO_COUNTRIES, TRADE_PAIRS

In [6]:
ENTSO_COUNTRIES.add('ES')

In [7]:
TRADE_PAIRS['FR'] = {'BE', 'CH', 'DE', 'IT', 'ES'}
TRADE_PAIRS['ES'] = {'FR', 'PT'}
TRADE_PAIRS['PT'] = {'ES'}

In [8]:
MRKT = "market for electricity, high voltage"

In [9]:
found = set([])

for act in ei:
    if act['name'] == MRKT and act['location'] in ENTSO_COUNTRIES:
        for exc in act.technosphere():
            if exc.input['unit'] == 'kilowatt hour' and "import from" not in exc.input['name']:
                found.add(exc.input['name'])

In [10]:
hv = found.copy()

In [11]:
mv = found.copy()

In [12]:
lv = found.copy()

In [13]:
hv

{'electricity production, deep geothermal',
 'electricity production, hard coal',
 'electricity production, hydro, pumped storage',
 'electricity production, hydro, reservoir, alpine region',
 'electricity production, hydro, reservoir, non-alpine region',
 'electricity production, hydro, run-of-river',
 'electricity production, lignite',
 'electricity production, natural gas, combined cycle power plant',
 'electricity production, natural gas, conventional power plant',
 'electricity production, nuclear, boiling water reactor',
 'electricity production, nuclear, pressure water reactor',
 'electricity production, nuclear, pressure water reactor, heavy water moderated',
 'electricity production, oil',
 'electricity production, peat',
 'electricity production, solar thermal parabolic trough, 50 MW',
 'electricity production, solar tower power plant, 20 MW',
 'electricity production, wind, 1-3MW turbine, offshore',
 'electricity production, wind, 1-3MW turbine, onshore',
 'electricity produ

## Mapping ENTSO to ecoinvent

In [14]:
ENTSO_MAPPING = {
    'Biomass': [
        'heat and power co-generation, wood chips, 6667 kW',
        'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014',
        'heat and power co-generation, biogas, gas engine',
    ],
    'Fossil Brown coal/Lignite': [
        'electricity production, lignite',
        'heat and power co-generation, lignite',
    ],
    'Fossil Coal-derived gas': [
        'treatment of coal gas, in power plant',
    ],
    'Fossil Gas': [
        'electricity production, natural gas, combined cycle power plant',
        'electricity production, natural gas, conventional power plant',
        'heat and power co-generation, natural gas, 500kW electrical, lean burn',
        'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical',
        'heat and power co-generation, natural gas, conventional power plant, 100MW electrical',
    ],
    'Fossil Hard coal': [
        'electricity production, hard coal',
        'heat and power co-generation, hard coal',
    ],
    'Fossil Oil': [
        'electricity production, oil',
        'heat and power co-generation, oil',
    ],
    'Fossil Oil shale': [
    ],
    'Fossil Peat': [
        'electricity production, peat',
    ],
    'Geothermal': [
        'electricity production, deep geothermal',
    ],
    # This will be a bit funky; could consider charging with average of last 48 hours?
    'Hydro Pumped Storage': [
        'electricity production, hydro, pumped storage',
    ],
    'Hydro Run-of-river and poundage': [
        'electricity production, hydro, run-of-river',
    ],
    'Hydro Water Reservoir': [
        'electricity production, hydro, reservoir, alpine region',
        'electricity production, hydro, reservoir, non-alpine region',
    ],
    'Nuclear': [
        'electricity production, nuclear, boiling water reactor',
        'electricity production, nuclear, pressure water reactor',
        'electricity production, nuclear, pressure water reactor, heavy water moderated',
    ],
    'Other': [
        'treatment of blast furnace gas, in power plant',
    ],
    'Other renewable': [
    ],
    'Solar': [
        'electricity production, photovoltaic, 3kWp facade installation, multi-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp facade installation, multi-Si, panel, mounted',
        'electricity production, photovoltaic, 3kWp facade installation, single-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp facade installation, single-Si, panel, mounted',
        'electricity production, photovoltaic, 3kWp flat-roof installation, multi-Si',
        'electricity production, photovoltaic, 3kWp flat-roof installation, single-Si',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, CIS, panel, mounted',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, CdTe, laminated, integrated',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, a-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, a-Si, panel, mounted',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, ribbon-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, ribbon-Si, panel, mounted',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, panel, mounted',
        'electricity production, photovoltaic, 570kWp open ground installation, multi-Si',
        'electricity production, solar thermal parabolic trough, 50 MW',
        'electricity production, solar tower power plant, 20 MW',        
    ],
    'Waste': [
        'electricity, from municipal waste incineration to generic market for electricity, medium voltage',
    ],
    'Wind Offshore': [
        'electricity production, wind, 1-3MW turbine, offshore',
    ],
    'Wind Onshore': [
        'electricity production, wind, 1-3MW turbine, onshore',
        'electricity production, wind, <1MW turbine, onshore',
        'electricity production, wind, >3MW turbine, onshore',
    ],
}

## ecoinvent constants

In [15]:
HIGH_VOLTAGE = {
    'electricity production, deep geothermal',
    'electricity production, hard coal',
    'electricity production, hydro, pumped storage',
    'electricity production, hydro, reservoir, alpine region',
    'electricity production, hydro, reservoir, non-alpine region',
    'electricity production, hydro, run-of-river',
    'electricity production, lignite',
    'electricity production, natural gas, combined cycle power plant',
    'electricity production, natural gas, conventional power plant',
    'electricity production, nuclear, boiling water reactor',
    'electricity production, nuclear, pressure water reactor',
    'electricity production, nuclear, pressure water reactor, heavy water moderated',
    'electricity production, oil',
    'electricity production, peat',
    'electricity production, wind, 1-3MW turbine, offshore',
    'electricity production, wind, 1-3MW turbine, onshore',
    'electricity production, wind, <1MW turbine, onshore',
    'electricity production, wind, >3MW turbine, onshore',
    'electricity production, solar thermal parabolic trough, 50 MW',
    'electricity production, solar tower power plant, 20 MW',    
    'heat and power co-generation, biogas, gas engine',
    'heat and power co-generation, hard coal',
    'heat and power co-generation, lignite',
    'heat and power co-generation, natural gas, 500kW electrical, lean burn',
    'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical',
    'heat and power co-generation, natural gas, conventional power plant, 100MW electrical',
    'heat and power co-generation, oil',
    'heat and power co-generation, wood chips, 6667 kW',
    'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014',
    'treatment of blast furnace gas, in power plant',
    'treatment of coal gas, in power plant',
}

MEDIUM_VOLTAGE = {
    'electricity, from municipal waste incineration to generic market for electricity, medium voltage',
}

LOW_VOLTAGE = {
    'electricity production, photovoltaic, 3kWp facade installation, multi-Si, laminated, integrated',
    'electricity production, photovoltaic, 3kWp facade installation, multi-Si, panel, mounted',
    'electricity production, photovoltaic, 3kWp facade installation, single-Si, laminated, integrated',
    'electricity production, photovoltaic, 3kWp facade installation, single-Si, panel, mounted',
    'electricity production, photovoltaic, 3kWp flat-roof installation, multi-Si',
    'electricity production, photovoltaic, 3kWp flat-roof installation, single-Si',
    'electricity production, photovoltaic, 3kWp slanted-roof installation, CIS, panel, mounted',
    'electricity production, photovoltaic, 3kWp slanted-roof installation, CdTe, laminated, integrated',
    'electricity production, photovoltaic, 3kWp slanted-roof installation, a-Si, laminated, integrated',
    'electricity production, photovoltaic, 3kWp slanted-roof installation, a-Si, panel, mounted',
    'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, laminated, integrated',
    'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted',
    'electricity production, photovoltaic, 3kWp slanted-roof installation, ribbon-Si, laminated, integrated',
    'electricity production, photovoltaic, 3kWp slanted-roof installation, ribbon-Si, panel, mounted',
    'electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, laminated, integrated',
    'electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, panel, mounted',
    'electricity production, photovoltaic, 570kWp open ground installation, multi-Si',
}

LOSSES = {
    'market for electricity, low voltage',
    'market for electricity, medium voltage',
    'market for electricity, high voltage',
}

TRANSFORMATION = {
    'electricity voltage transformation from medium to low voltage',
    'electricity voltage transformation from high to medium voltage',
}

IGNORE = {
    'market group for electricity, high voltage',
}

## Comparing aggregate values

In [16]:
from bentso import CachingDataClient as CDC

In [17]:
cdc = CDC()

Using data directory /Users/cmutel/Code/akula/entso-data-cache


In [18]:
def clean_consumption_values(df):
    df.drop([col for col in df.columns if col[1] == 'Actual Consumption'], axis=1, inplace=True)
    df.columns = df.columns.get_level_values(0)
    return df

In [19]:
def get_df(country, year=2020):
    df = cdc.get_generation(country=country, year=2020)
    if isinstance(df.columns, pd.core.indexes.multi.MultiIndex):
        df = clean_consumption_values(df)
    return df

In [20]:
def included_exchange(exc):
    return (
        (exc.input != exc.output) and
        (exc.input['unit'] == 'kilowatt hour') and
        ("import from" not in exc.input['name']) and
        (exc.input['name'] not in IGNORE) and
        (exc.input['name'] not in TRANSFORMATION)
    )

### Dealing with high, medium, and low voltage mixes

ENTSO gives us totals generated in MWh; we need to translate that to relative shares of the separat markets, while keeping the totals correct.

The low voltage market is made up of PV and imports from medium voltage; similarly, the medium voltage is waste incineration and imports from high voltage. There are also losees in transmission and distribution, but we ignore these in this calculation.

The easiest to gets the numbers right is to start from low voltage (LV). Let's assume it is 100 MWh total. Then the total generation allocated to the aggregated LV generations sources is:

$\frac{LV}{TOTAL}$

And the import from MV is:

$\frac{1 - LV}{TOTAL}$

Medium voltage is already a bit trickier. We need to keep the absolute and relative numbers in our heads simultaneously. We want the fraction of the generation that MV sources can contribute as a fraction of the total MV market, which is MV plus HV, or TOTAL - LV. The scaling factor then applied to MV sources is then:

$\frac{MV}{TOTAL - LV}$

And the import from HV is:

$\frac{1 - MV}{TOTAL - LV}$

For HV, we can just normalize all HV sources to sum to one; there is no import from a higher voltage level.

However, we also need to include trade! So these equations are OK for comparing generation, we would need to substitute $IMPORTS + TOTAL$ for $TOTAL$ in each equation in actual use.

In [21]:
def compare_fractional_generation(country, year=2020):
    hv = ei.get(name='market for electricity, high voltage', location=country)
    mv = ei.get(name='market for electricity, medium voltage', location=country)
    lv = ei.get(name='market for electricity, low voltage', location=country)
    
    entso_df = get_df(country=country, year=2020).sum(axis=0)
    entso_df = (entso_df / entso_df.sum()).to_dict()
    
    sum_hv = sum(exc['amount'] for exc in hv.technosphere() if included_exchange(exc))
    # sum_mv = sum(exc['amount'] for exc in mv.technosphere() if included_exchange(exc))
    # sum_lv = sum(exc['amount'] for exc in lv.technosphere() if included_exchange(exc))
    conv_mv_to_lv = sum(exc['amount'] for exc in lv.technosphere() 
                        if exc.input['name'] == 'electricity voltage transformation from medium to low voltage')
    conv_hv_to_lv = (conv_mv_to_lv *
                     sum(exc['amount'] for exc in mv.technosphere() 
                         if exc.input['name'] == 'electricity voltage transformation from high to medium voltage')
                    )
    
    ei_data = {
        'Solar': sum(exc['amount'] for exc in lv.technosphere() if included_exchange(exc)),
        'Waste': sum(exc['amount'] for exc in mv.technosphere() if included_exchange(exc)) * conv_mv_to_lv,
    }
    
    for key, lst in ENTSO_MAPPING.items():
        if key in ei_data:
            continue
        exchanges = [exc for exc in hv.technosphere() if exc.input['name'] in lst]
        if not exchanges:
            continue
        
        ei_data[key] = (
            sum(exc['amount'] for exc in exchanges) 
            / sum_hv  # production doesn't add up to 1 - there are also imports
            * conv_hv_to_lv
        )
    
    all_keys = set(ei_data).union(set(entso_df))
    for key in all_keys:
        if key not in entso_df:
            entso_df[key] = 0
        if key not in ei_data:
            ei_data[key] = 0
    
    return pd.DataFrame({'ENTSO': entso_df, 'ecoinvent': ei_data})

In [29]:
(get_df('ES', 2019).sum(axis=0)).sum() / 1000

238652.11

In [31]:
df = compare_fractional_generation('ES', 2019)
df

,ENTSO,ecoinvent
Biomass,0.016056,0.012940
Fossil Brown coal/Lignite,0.000000,0.007119
Fossil Coal-derived gas,0.000000,0.000467
Fossil Gas,0.257964,0.120847
Fossil Hard coal,0.021526,0.147656
Fossil Oil,0.007724,0.040121
Fossil Oil shale,0.000000,0.000000
Fossil Peat,0.000000,0.000000
Geothermal,0.000000,0.000000
Hydro Run-of-river and poundage,0.042326,0.093979


Data read manually from https://www.iea.org/countries/spain

In [34]:
df['IEA'] = pd.Series({
    'Biomass': 4802,
    'Fossil Brown coal/Lignite': 0,
    'Fossil Coal-derived gas': 0,
    'Fossil Gas': 83703,
    'Fossil Hard coal': 13982,
    'Fossil Oil': 12883,
    'Fossil Oil shale': 0,
    'Fossil Peat': 0,
    'Geothermal': 0,
    'Hydro Run-of-river and poundage': 26874,
    'Hydro Water Reservoir': 0,
    'Marine': 0,
    'Nuclear': 58349,
    'Other': 139,
    'Other renewable': 0,
    'Solar': 9420,
    'Waste': 1755,
    'Wind Offshore': 0,
    'Wind Onshore': 55647,
    'Hydro Pumped Storage': 0,
})

In [35]:
df['IEA'] = df['IEA'] / df['IEA'].sum()

In [45]:
import requests

In [52]:
URL = "https://apidatos.ree.es/en/datos/generacion/estructura-generacion?start_date=2019-01-01T00:00&end_date=2019-12-31T23:59&time_trunc=year"

In [53]:
res = {o['attributes']['title']: o['attributes']['values'][0]['percentage'] for o in requests.get(URL).json()['included']}
res

{'Hydro': 0.09477104131458193,
 'Pumped storage': 0.00630875586852493,
 'Nuclear': 0.21402632991933143,
 'Coal': 0.04857837137561619,
 'Fuel + Gas': 3.8339327256957146e-12,
 'Diesel engines': 0.01087323696059382,
 'Gas turbine': 0.002570774873639019,
 'Steam turbine': 0.008392519636942236,
 'Combined cycle': 0.2117939974506982,
 'Hydroeolian': 8.913402077067102e-05,
 'Wind': 0.20797189501836943,
 'Solar photovoltaic': 0.03547161796662054,
 'Thermal solar': 0.01980774944186908,
 'Other renewables': 0.013870071713414273,
 'Cogeneration': 0.11354224357492992,
 'Non-renewable waste': 0.008520774804365222,
 'Renewable waste': 0.003411486055899025,
 'Total generation': 1}

In [58]:
res_mapping = {
    "Hydro": "Hydro Run-of-river and poundage",
    "Pumped storage": "Hydro Run-of-river and poundage",
    "Nuclear": "Nuclear",
    "Coal": "Fossil Hard coal",
    "Fuel + Gas": "Fossil Oil",
    "Diesel engines": "Fossil Oil",
    "Gas turbine": "Fossil Gas",
    "Steam turbine": "Fossil Gas",
    "Combined cycle": "Fossil Gas",
    "Hydroeolian": "Wind Onshore",
    "Wind": "Wind Onshore",
    "Solar photovoltaic": "Solar",
    "Thermal solar": "Solar",
    "Other renewables": "Other renewable",
    "Cogeneration": "Other",
    "Non-renewable waste": "Waste",
    "Renewable waste": "Waste",
}

In [62]:
res_series = {label: 0 for label in df.index}

for key, value in res.items():
    try:
        res_series[res_mapping[key]] += value
    except KeyError:
        print(key)
    
df["RES"] = pd.Series(res_series)

Total generation


In [63]:
df

,ENTSO,ecoinvent,IEA,RES
Biomass,0.016056,0.012940,0.017948,0.000000
Fossil Brown coal/Lignite,0.000000,0.007119,0.000000,0.000000
Fossil Coal-derived gas,0.000000,0.000467,0.000000,0.000000
Fossil Gas,0.257964,0.120847,0.312845,0.222757
Fossil Hard coal,0.021526,0.147656,0.052259,0.048578
Fossil Oil,0.007724,0.040121,0.048151,0.010873
Fossil Oil shale,0.000000,0.000000,0.000000,0.000000
Fossil Peat,0.000000,0.000000,0.000000,0.000000
Geothermal,0.000000,0.000000,0.000000,0.000000
Hydro Run-of-river and poundage,0.042326,0.093979,0.100443,0.101080


In [65]:
df.to_excel("Spain.xlsx")

In [93]:
from pandas import ExcelWriter

In [98]:
with pd.ExcelWriter("generation.xlsx") as writer:
    for country in ENTSO_COUNTRIES:
        df = compare_fractional_generation(country, 2019) 
        df['difference'] = df['ENTSO'] - df['ecoinvent']
        df.to_excel(writer, sheet_name=country)

## Generate a datapackage to substitute ENTSO data for ecoinvent numbers

We will only change the basic high, medium, and low voltage markets. Industry- or customer-specific electricity markets are left alone.

### Changing the way electricity is imported

To model electricity imports, ecoinvent uses the following chain:

    electricity, high voltage (foo) 
        <- electricity, high voltage, import from bar
            <- electricity, high voltage (bar)
            
We change simplify this. There is no need to create this proxy process, we can directly use `electricity, high voltage (bar)`.

    electricity, high voltage (foo) 
        <- electricity, high voltage (bar)